In [ ]:
!pip3 install fuzzywuzzy

In [1]:
!pip install meteostat

In [218]:
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from meteostat import Point, Daily
import pandas as pd
from bs4 import BeautifulSoup
import requests
import sqlite3

## Table location(resort_name, latitude, longitude, location_catalog, state, city, zipcode, address, url)

In [339]:
# Extract relevant information
url = "https://en.m.wikipedia.org/wiki/List_of_ski_areas_and_resorts_in_the_United_States"

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the HTML content of the page
    soup = BeautifulSoup(response.content, 'html.parser')
    # print(soup)
    sections = soup.find_all(['h2', 'h3'])
    data = []
    str_data = []

    location_catalog = ''  

    for section in sections:
        if section.name == 'h2':
            headline = section.find('span', {'class': 'mw-headline'})
            if headline and ' (' in headline.text.strip():
                #print(headline.text.strip())
                location_catalog = headline.text.strip()
                location_catalog = location_catalog.split(' (')[0] 
                #print(location_catalog)
        elif section.name == 'h3' and 'mw-headline' in section.span.attrs.get('class', []):
            state = section.span.a.get('title')
            if ' (' in state:
                state = state.split(' (')[0] 
            resort_list = section.find_next('ul')
            resorts = resort_list.find_all('li')
            #print(resorts)
            for resort in resorts:
                if resort:
                    resort_name = resort.text.strip()
                    resort_name = resort_name.split(' (')[0] if ' (' in resort_name else resort_name
                    #print(resort_name)
                    data.append({'location_catalog': location_catalog, 'state': state, 'resort_name': resort_name})
                    str_data.append(str(resort_name))
                
    # Create DataFrame
    ski_df = pd.DataFrame(data)
    #print(df)

In [90]:
ski_df

,location_catalog,state,resort_name
0,New England,Connecticut,Mohawk Mountain Ski Area — Cornwall
1,New England,Connecticut,Mount Southington — Plantsville
2,New England,Connecticut,Powder Ridge Ski Area — Middlefield
3,New England,Connecticut,Ski Sundown — New Hartford
4,New England,Maine,Baker Mountain — Bingham
...,...,...,...
461,West Coast,Washington,Alpental
462,West Coast,Washington,Summit Central
463,West Coast,Washington,Summit East
464,West Coast,Washington,Summit West


In [98]:
ski_df.groupby(["location_catalog","state"]).count()

resort_name
location_catalog state                      
Mid-Atlantic     Maryland                  1
                 New Jersey                4
                 New York                 36
                 Pennsylvania             22
Midwest          Illinois                  5
                 Indiana                   2
                 Iowa                      4
                 Michigan                 34
                 Minnesota                17
                 Missouri                  2
                 North Dakota              3
                 Ohio                      5
                 South Dakota              3
                 Wisconsin                36
New England      Connecticut               4
                 Maine                    19
                 Massachusetts            12
                 New Hampshire            29
                 Rhode Island              1
                 Vermont                  27
Rocky Mountains  Arizona                   4
                 Colorado                 33
                 Idaho                    19
                 Montana                  16
                 New Mexico                9
                 Utah                     15
                 Wyoming                  11
Southeast        Alabama                   1
                 North Carolina            6
                 Tennessee                 1
                 Texas                     1
                 Virginia                  4
                 West Virginia             5
West Coast       Alaska                    8
                 California               29
                 Nevada                    5
                 Oregon                   12
                 Washington               21

In [86]:
# Function to get location details from Google Maps API
def get_location_details(resort_name, api_key):
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {"address": resort_name, "key": api_key}

    response = requests.get(base_url, params=params)
    data = response.json()

    if data["status"] == "OK":
        result = data["results"][0]
        location = result["geometry"]["location"]
        address_components = result["address_components"]

        # Extracting relevant information
        latitude, longitude = location["lat"], location["lng"]
        address = result.get("formatted_address", "")
        state = next((component["long_name"] for component in address_components if "administrative_area_level_1" in component["types"]), "")
        city = next((component["long_name"] for component in address_components if "locality" in component["types"]), "")
        zipcode = next((component["long_name"] for component in address_components if "postal_code" in component["types"]), "")
        address = result.get("formatted_address", "")
        
        elevation_url = f'https://maps.googleapis.com/maps/api/elevation/json?locations={latitude},{longitude}&key={api_key}'
        elevation_response = requests.get(elevation_url)
        elevation_data = elevation_response.json()

        if elevation_data["status"] == "OK" and elevation_data.get("results"):
            elevation = elevation_data["results"][0].get("elevation")
        else:
            elevation = None

        #Use Places API to get additional details
        places_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"
        places_params = {
            "input": resort_name,
            "inputtype": "textquery",
            "fields": "place_id",
            "key": api_key
        }

        places_response = requests.get(places_url, params=places_params)
        places_data = places_response.json()

        if places_data.get("status") == "OK" and places_data.get("candidates"):
            place_id = places_data["candidates"][0]["place_id"]

            # Use Place Details API to get website
            details_url = f'https://maps.googleapis.com/maps/api/place/details/json?place_id={place_id}&key={api_key}'
            details_response = requests.get(details_url)
            details_data = details_response.json()
            url = details_data["result"].get("website", "")

            return latitude, longitude, elevation, state, city, zipcode, address, url
        else:
            return latitude, longitude, elevation, state, city, zipcode, address, None
    else:
        return None, None, None, None, None, None, None, None


In [92]:
db_file = "skiDataset.db"
conn = sqlite3.connect(db_file)
cursor = conn.cursor()
cursor.execute('''
    CREATE TABLE IF NOT EXISTS location (
        resort_name TEXT PRIMARY KEY,
        latitude REAL,
        longitude REAL,
        elevation REAL,
        location_catalog TEXT,
        state TEXT,
        city TEXT,
        zipcode TEXT,
        address TEXT,
        url TEXT
    )
''')

# get the latitude, longtitude from google maps API and insert into databse
api_key = 'HIDDEN'
for index, row in ski_df.iterrows():
    location_catalog = row['location_catalog']
    resort_name = row['resort_name']
    latitude, longitude, elevation, state, city, zipcode, address, url = get_location_details(resort_name, api_key)
    cursor.execute('''
    INSERT INTO location (resort_name, latitude, longitude, elevation, location_catalog, state, city, zipcode, address, url)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
''', (resort_name, latitude, longitude, elevation, location_catalog, state, city, zipcode, address, url))
    
# Commit the changes and close the connection
conn.commit()
conn.close()

## Table weather (resort_name, latitude, longitude, date, tavg, tmin, tmax, snowfall)

In [139]:
import requests
from datetime import datetime, timedelta

def getWeather(resort_name, latitude, longitude, cursor):
    start_date = datetime(2014, 1, 1)
    end_date = datetime(2024, 2, 24)
    access_key = "hidden"
    coordinate = f"{latitude},{longitude}"

    while start_date < end_date:
        # Set the end date for each 60-day interval
        interval_end_date = start_date + timedelta(days=59)
        if interval_end_date > end_date:
            interval_end_date = end_date

        params = {
            'access_key': access_key,
            'query': coordinate,
            'historical_date_start': start_date.strftime('%Y-%m-%d'),
            'historical_date_end': interval_end_date.strftime('%Y-%m-%d'),
            'unit': 'm'
        }

        api_result = requests.get('http://api.weatherstack.com/historical', params)
        api_response = api_result.json()
        if "success" in api_response:
            print(api_response)
        past_data = dict(api_response["historical"])

        for date, info in past_data.items():
            info = dict(info)
            tmin = info["mintemp"]
            tmax = info["maxtemp"]
            tavg = info["avgtemp"]
            snowfall = info["totalsnow"] * 10  # cm to mm

            cursor.execute('''
                INSERT INTO weather(resort_name, latitude, longitude, date, tavg, tmin, tmax, snowfall)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?)
            ''', (resort_name, latitude, longitude, date, tavg, tmin, tmax, snowfall))

        # Move to the next 60-day interval
        start_date = interval_end_date + timedelta(days=1)


In [140]:
conn = sqlite3.connect(db_file) 
cursor = conn.cursor()

# Create the table weather
cursor.execute('''
    CREATE TABLE IF NOT EXISTS weather (
        resort_name TEXT,
        latitude REAL,
        longitude REAL,
        date TEXT,
        tavg REAL,
        tmin REAL,
        tmax REAL,
        snowfall REAL,
        PRIMARY KEY (latitude, longitude, date),
        FOREIGN KEY (latitude, longitude) REFERENCES location (latitude, longitude),
        FOREIGN KEY (resort_name) REFERENCES location (resort_name)
    )
''')
# Get the data
cursor.execute('SELECT resort_name, latitude, longitude FROM location;')
result_set = cursor.fetchall()

# Process the data 
for row in result_set:
    resort_name = row[0]
    latitude = row[1]
    longitude = row[2]
    #print(resort_name, latitude, longitude)
    getWeather(resort_name, latitude, longitude, cursor)
#print(result_df)

# Commit changes and close the connection
conn.commit()
conn.close()


KeyboardInterrupt: 

In [183]:
len(unfinished_ski_resort)

358

In [273]:
def getWeather2(resort_name, latitude, longitude, elevation, cursor):
    start_date = datetime(2014, 1, 1)
    end_date = datetime(2024, 2, 24)
    # Create Point 
    point = Point(round(latitude), round(longitude), round(elevation))
    # Get daily data 
    data = Daily(point, start_date, end_date)
    data = data.fetch()[["tavg", "tmin", "tmax", "prcp"]].fillna(0)
    print(data)
    for date, row in data.iterrows():
        tmin = row["tmin"]
        tmax = row["tmax"]
        tavg = row["tavg"]
        snowfall = row["prcp"]* 13 if tavg < 0 else 0
        print(date, tmin, tmax, tavg, snowfall)
        cursor.execute('''
            INSERT OR REPLACE INTO weather(resort_name, latitude, longitude, date, tavg, tmin, tmax, snowfall)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        ''', (resort_name, latitude, longitude, date.strftime('%Y-%m-%d'), tavg, tmin, tmax, snowfall))

In [274]:
db_file = "skiDataset.db"
unfinished = 358
# Connect to the database
conn = sqlite3.connect(db_file)
cursor = conn.cursor()
cursor.execute('SELECT resort_name, latitude, longitude, elevation FROM location;')
result_set = cursor.fetchall()
try:
    for ski_resort in result_set:
        resort_name = ski_resort[0]
        latitude = ski_resort[1]
        longitude = ski_resort[2]
        elevation = ski_resort[3]
        if ski_resort not in list(pd.DataFrame(finish_data)[0]) and resort_name and latitude and longitude and elevation:
            getWeather2(resort_name, latitude, longitude, elevation, cursor)
            print("finished gathering weather data for ", resort_name)
            unfinished = unfinished - 1
            print("still need to run: ", unfinished)
finally:
    conn.commit()
    conn.close()

Empty DataFrame
Columns: [tavg, tmin, tmax, prcp]
Index: []
finished gathering weather data for  Whitetail Ski Resort — Mercersburg
still need to run:  357


## Clean the data (Google map result given NULL coordinate)

In [342]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('./skiDataset.db')
cursor = conn.cursor()

cursor.execute('''
     SELECT *
     FROM location
     WHERE latitude IS NULL OR longitude IS NULL
''')
data = cursor.fetchall()

for resort in data:
    resort_name = resort[0]

    # Delete rows from the 'location' table
    cursor.execute('''
        DELETE FROM location
        WHERE resort_name = ?
    ''', (resort_name,))

    # Delete rows from the 'weather' table
    cursor.execute('''
        DELETE FROM weather
        WHERE resort_name = ?
    ''', (resort_name,))

# Commit the changes and close the connection
conn.commit()
conn.close()

## Table skiInfo (resort_name, rank, rating, image_url, state)

In [209]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

resort_names = []
rankings = []
ratings = []
image_urls = []
states = []
for i in range(5):
    if i == 0:
        url = "https://www.skiresort.info/best-ski-resorts/usa/"
    else:
        num = i+1
        url = f"https://www.skiresort.info/best-ski-resorts/usa/page/{num}/"
    # Send a GET request to the URL
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    resort_items = soup.find_all('div', class_="resort-list-item")


    for resort_item in resort_items:
        # Get resort name
        resort_name_tag = resort_item.find('a', class_='h3') if resort_item.find('a', class_='h3') else None
        resort_name = resort_name_tag.text.strip().split('.')[1] if resort_name_tag else None

        # Get ranking
        ranking = resort_name_tag.text.strip().split('.')[0] if resort_name_tag else None

        # Get rating
        rating_tag = resort_item.find('div', class_='rating-list')
        rating = rating_tag["title"].strip().split()[0] if rating_tag else None

        # Get image URL
        img_tag = resort_item.find('div', class_='resort-list-item-img-wrap').find('img') if resort_item.find('div', class_='resort-list-item-img-wrap') else None
        img_url = "https://www.skiresort.info" + img_tag['data-src'] if img_tag else None
        
        # Get the state
        sub_breadcrumb_div = soup.find('div', class_='sub-breadcrumb')
        state_tag = sub_breadcrumb_div.find_all('a')[-1]
        state = state_tag.text.strip() if state_tag else None
        
        # Append data to lists
        resort_names.append(resort_name)
        rankings.append(ranking)
        ratings.append(rating)
        image_urls.append(img_url)
        states.append(state)

# Create DataFrame
skiInfo_data = {
    'resort_name': resort_names,
    'rank': rankings,
    'rating': ratings,
    'image_url': image_urls,
    'state': states
}

skiInfo_df = pd.DataFrame(skiInfo_data)

# Print DataFrame
print(skiInfo_df)


                                           resort_name rank rating  \
0                                                 Vail    1    4.6   
1                                            Telluride    2    4.5   
2                                         Beaver Creek    2    4.5   
3                                          Deer Valley    4    4.4   
4                                            Park City    4    4.4   
..                                                 ...  ...    ...   
523        Sand Master Park – Florence (sand ski area)  483    1.8   
524                                Rocking Horse Ranch  483    1.8   
525   Inside Ski Training Center – Leesburg (dry sl...  483    1.8   
526            Infinity Slopes – Loveland (dry slopes)  483    1.8   
527                                  Eagle Island Park  483    1.8   

                                             image_url     state  
0    https://www.skiresort.info/fileadmin/_processe...  Colorado  
1    https://www.skiresor

In [216]:
conn = sqlite3.connect('skiDataset.db')  
cursor = conn.cursor()

# Create the skiInfo table
create_table_query = '''
CREATE TABLE IF NOT EXISTS skiInfo (
    resort_name TEXT PRIMARY KEY,
    rank INTEGER,
    rating REAL,
    image_url TEXT,
    state TEXT
);
'''
cursor.execute(create_table_query)

# Insert data into the skiInfo table
insert_data_query = '''
INSERT OR REPLACE INTO skiInfo (resort_name, rank, rating, image_url, state) VALUES
    (?, ?, ?, ?, ?);
'''
for index,row in skiInfo_df.iterrows():
    resort_name, rank, rating, image_url, state = row
    #print(resort_name, rank, rating, image_url, state)
    cursor.execute(insert_data_query, (resort_name, rank, rating, image_url, state))

# Commit the changes and close the connection
conn.commit()
conn.close()

## Table name_mapping(location_name, skiinfo_name)

In [328]:
db_file = "skiDataset.db"
conn = sqlite3.connect(db_file)
cursor = conn.cursor()
cursor.execute('''
     SELECT resort_name, state
     FROM location
''')
data_l = cursor.fetchall()
df_l = pd.DataFrame(data_l)
cursor.execute('''
     SELECT resort_name, state
     FROM skiInfo
''')
data_s = cursor.fetchall()
df_s = pd.DataFrame(data_s)

# Create a mapping table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS name_mapping (
        location_name TEXT PRIMARY KEY,
        skiinfo_name TEXT
    )
''')
name_dict = {}
for index,row_l in df_l.iterrows():
    name_l = row_l[0]
    #print(state_l)
    for i,row_s in df_s.iterrows():
        name_s = row_s[0]
        similarity_ratio = fuzz.ratio(name_l, name_s)
        threshold = 60
        if similarity_ratio > threshold:
            name_dict[name_l] = name_s
            cursor.execute('''
                INSERT INTO name_mapping (location_name, skiinfo_name)
                VALUES (?, ?)
            ''', (name_l, name_s)) 
            break
conn.commit()
conn.close()

In [315]:
len(name_dict)

226

In [314]:
name_dict

{'Black Mountain of Maine — Rumford': ' Black Mountain – Rumford',
 'Camden Snow Bowl — Camden': ' Camden Snow Bowl',
 'Hermon Mountain — Hermon': ' Homewood Mountain Resort',
 'Lonesome Pine Trails — Fort Kent': ' Lonesome Pine Trails – Fort Kent',
 'Lost Valley — Auburn': ' Lost Valley',
 'Pinnacle Ski Club — Pittsfield': ' Pinnacle Park – Pittsfield',
 'Pleasant Mountain — Bridgton': ' Pleasant Mountain',
 'Powderhouse Hill — South Berwick': ' Powderhouse Hill – South Berwick',
 'Quoggy Jo — Presque Isle': ' Quoggy Jo Ski Center – Presque Isle',
 'Sunday River — Newry': ' Sunday River',
 'Titcomb Mountain — Farmington': ' Mystic Mountain – Farmington',
 'Blue Hills Ski Area — Canton': ' Buck Hill Ski Area',
 'Jiminy Peak — Hancock': ' Jiminy Peak',
 'Mount Greylock Ski Club — Williamstown': ' Mount Greylock – South Williamstown',
 'Otis Ridge — Otis': ' Otis Ridge',
 'Ski Ward — Shrewsbury': ' Ward Hill – Shrewsbury',
 'Wachusett Mountain — Princeton': ' Wachusett Mountain',
 'Black

In [331]:
import sqlite3
import pandas as pd

# Connect to the database
conn = sqlite3.connect("skiDataset.db")
cursor = conn.cursor()

# Create a mapping table
cursor.execute('''
SELECT *
FROM location
INNER JOIN name_mapping ON location.resort_name = name_mapping.location_name
INNER JOIN skiInfo ON name_mapping.skiInfo_name = skiInfo.resort_name
ORDER by skiInfo.rank;
''')

# Fetch the data
data = cursor.fetchall()

# Create a DataFrame
df = pd.DataFrame(data)

# Print the DataFrame
print(df)

# Close the connection
conn.close()


                                           0          1           2   \
0                     Deer Valley — Park City  40.622608 -111.485089   
1            Mammoth Mountain — Mammoth Lakes  37.630769 -119.032634   
2            Palisades Tahoe — Olympic Valley  39.190609 -120.248391   
3                        Sunday River — Newry  44.473418  -70.856873   
4                 Sunrise Park Resort — Greer  33.973727 -109.564228   
..                                        ...        ...         ...   
221                      Nutt Hill — Plymouth  43.748605  -87.977038   
222                      Powers Bluff — Arpin  44.526171  -90.065957   
223            Standing Rocks — Stevens Point  44.431244  -89.396241   
224  Steamboat Ski Resort — Steamboat Springs  40.453798 -106.770880   
225        Leavenworth Ski Hill — Leavenworth  47.614112 -120.668688   

              3                4           5                  6      7   \
0    2534.705811  Rocky Mountains        Utah          Park 